# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [13]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [14]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,pangody,65.8500,74.4859,-9.64,92,100,5.21,RU,1681843455
1,1,kodiak,57.7900,-152.4072,1.73,48,0,5.66,US,1681843455
2,2,new norfolk,-42.7826,147.0587,12.72,82,100,0.45,AU,1681843455
3,3,stanley,54.8680,-1.6985,8.49,81,0,4.63,GB,1681843321
4,4,sao vicente,-23.9631,-46.3919,27.75,60,0,2.24,BR,1681843456


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot1 = city_data_df.hvplot.points("Lng","Lat",geo=True,tiles="EsriImagery",size= "Humidity",scale= .45, color= "City", alpha=.75)

# Display the map
map_plot1

ImportError: Geographic projection support requires GeoViews and cartopy.

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values

#A max temperature lower than 27 degrees but higher than 21
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

#Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

#Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop results with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
62,62,illizi,26.4833,8.4667,26.08,13,0,2.79,DZ,1681843470
77,77,saint-pierre,-21.3393,55.4781,24.82,73,0,2.57,RE,1681843473
82,82,bhamo,24.2667,97.2333,23.91,30,0,0.84,MM,1681843473
98,98,myitkyina,25.3833,97.4000,23.59,35,0,1.43,MM,1681843477
162,162,marsa alam,25.0757,34.8918,26.98,40,0,2.40,EG,1681843488
163,163,vallenar,-28.5708,-70.7581,25.99,26,0,3.23,CL,1681843488
168,168,mogok,22.9167,96.5000,22.50,33,0,1.00,MM,1681843489
216,216,valparaiso,-33.0393,-71.6273,25.13,37,0,4.12,CL,1681843213
265,265,gebeit,21.0667,36.3167,24.27,20,0,2.26,SD,1681843515
290,290,katsuren-haebaru,26.3369,127.8719,23.01,83,0,4.12,JP,1681843521


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
62,illizi,DZ,26.4833,8.4667,
77,saint-pierre,RE,-21.3393,55.4781,
82,bhamo,MM,24.2667,97.2333,
98,myitkyina,MM,25.3833,97.4000,
162,marsa alam,EG,25.0757,34.8918,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel

categories = "accommodation.hotel"

radius = 10000
params = {
    "categories":categories,
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
illizi - nearest hotel: بوناقة
saint-pierre - nearest hotel: Tropic Hotel
bhamo - nearest hotel: Winlight Hotel
myitkyina - nearest hotel: New Light Hotel
marsa alam - nearest hotel: Blue House Hotel
vallenar - nearest hotel: Hotel del Marqués
mogok - nearest hotel: Mogok Hill
valparaiso - nearest hotel: Ibis Hotel
gebeit - nearest hotel: No hotel found
katsuren-haebaru - nearest hotel: No hotel found
mcrae - nearest hotel: No hotel found
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
carolina beach - nearest hotel: Carolina Beach Inn
galliano - nearest hotel: Fairfield Inn & Suites by Marriott Cut Off-Galliano
holualoa - nearest hotel: Kona Hotel
kailua-kona - nearest hotel: Kona Seaside Hotel
khawr fakkan - nearest hotel: Raynor Hotel
saint-joseph - nearest hotel: PALM Hotel & Spa
batouri - nearest hotel: No hotel found
tshabong - nearest hotel: No hotel found


,City,Country,Lat,Lng,Hotel Name
62,illizi,DZ,26.4833,8.4667,بوناقة
77,saint-pierre,RE,-21.3393,55.4781,Tropic Hotel
82,bhamo,MM,24.2667,97.2333,Winlight Hotel
98,myitkyina,MM,25.3833,97.4000,New Light Hotel
162,marsa alam,EG,25.0757,34.8918,Blue House Hotel
163,vallenar,CL,-28.5708,-70.7581,Hotel del Marqués
168,mogok,MM,22.9167,96.5000,Mogok Hill
216,valparaiso,CL,-33.0393,-71.6273,Ibis Hotel
265,gebeit,SD,21.0667,36.3167,No hotel found
290,katsuren-haebaru,JP,26.3369,127.8719,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

ImportError: Geographic projection support requires GeoViews and cartopy.